#### **프롬프트(Prompt)**

* 프롬프트 단계 개요  
    * 검색기에서 찾아낸 여러 문서(컨텍스트)를 바탕으로 언어 모델이 최종 답변을 생성하기 위해 사용하는 **질문이나 명령문을 만드는 과정**  
    * RAG 시스템에서 매우 중요한 단계

* 프롬프트의 필요성  
  * **문맥(Context) 설정**  
    *  모델이 특정 상황·정보를 이해하고 활용하도록 도움  
    *  제공된 문서를 효과적으로 반영하여 답변의 정확성과 연관성 향상  

  * **정보 통합**  
    * 검색된 여러 문서의 서로 다른 내용이나 관점을 합쳐 일관된 형식으로 정리  
    * 모델이 혼란 없이 여러 정보를 같이 활용할 수 있게 함  

  * **응답 품질 향상**  
    * 프롬프트가 잘 구성될수록 모델이 더 정확하고 유용한 답변 생성  
    * 덜 구조화되거나 모호한 프롬프트는 부정확하거나 엉뚱한 응답 가능성 증가

* **RAG 프롬프트 기본 구조**
  
    `지시사항(Instruction)`
    `질문(사용자 입력 질문)`
    `문맥(검색된 정보)`
    `예시)  `
    `당신은 질문-답변 AI 어시스턴트입니다.`  
    `검색된 문맥(context)를 활용하여 질문(question)에 답하세요.`  
    `문맥에 답이 없으면 '모른다'고 답해주세요. 한국어로 대답하세요.`

    `# Question: {사용자 질문}`  
    `# Context: {검색된 정보}`

* 프롬프트 중요성 요약  
  * RAG에서 프롬프트는 LLM이 최적의 성능을 내도록 하는 핵심 데이터 인터페이스  
  * 잘 짜인 프롬프트가 없으면, LLM이 검색된 정보를 제대로 이해·활용 못해 부정확한 답변 생성  
  * 결과적으로 사용자 만족도와 시스템 신뢰도에 큰 영향  
  * 따라서 프롬프트 설계에 신중을 기하고 반복 개선이 필요

---

<small>

* 기본 환경 설정

In [ ]:
# 기본 모듈 임포트
import os
import asyncio
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

In [ ]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable         # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 
  * 셀 출력
      * --- LangSmith 환경 변수 확인 ---
      * ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
      * ✅ LangSmith 프로젝트: 'L**************'
      * ✅ LangSmith API Key: 설정됨
      *   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

In [ ]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트


print("\n--- LangChain 체인 설정 ---")                                # --- LangChain 체인 설정 ---

In [ ]:
# LLM 객체 정의하기

# model = gemini-1.5-flash            -> 기본으로 사용 중
# model2 = gemini-2.5-falsh-lite

try:
    model = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-1.5-flash",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ Google GenAI 모델 초기화 성공.

In [ ]:
# model2 = gemini-2.5-falsh-lite

try:
    model2 = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )
    print("✅ gemini-2.5-flash-lite 호출 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ gemini-2.5-flash-lite 호출 성공.

-----

#### (1) **`from_template()`** 메소드 → **PromptTemplate 객체** 생성

* 치환될 변수 = `{변수}` -> 템플릿 정의하기

In [ ]:
# 템플릿 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template() → PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

# PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

* `{country}`에 값을 넣어 문장 생성해보기

In [ ]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt                                                  # '대한민국의 수도는 어디인가요?'

* `from_template()`로 만든 `PrompotTemplate` 객체 + `LLM` = `Chain`

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain_prompt1 = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain_prompt1.invoke("대한민국").content                            # '대한민국의 수도는 **서울**입니다.'

---

#### (2) `PromptTemplate 객체` 생성과 **동시에 `prompt` 생성**

* 추가 유효성 검사를 위해 **`input_variables`를 명시적으로 지정**하기

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt                      

# PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [ ]:
# prompt 생성
prompt.format(country="대한민국")                   # '대한민국의 수도는 어디인가요?'

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"                                      # dictionary 형태로 partial_variables를 전달
    },
)

prompt

<small>

* 
  * 셀 출력
      * PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [ ]:
prompt.format(country1="대한민국")                             # '대한민국과 미국의 수도는 각각 어디인가요?'

In [ ]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial                                              

<small>

* 
  * 셀 출력
      * PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [ ]:
prompt_partial.format(country1="대한민국")                      # '대한민국과 캐나다의 수도는 각각 어디인가요?'

In [ ]:
chain = prompt_partial | model
chain.invoke("대한민국").content                                # '대한민국의 수도는 **서울**이고, 캐나다의 수도는 **오타와**입니다.'

In [ ]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

# '대한민국의 수도는 **서울**이고, 호주의 수도는 **캔버라**입니다.'

---

#### (3) **`partial_variable`** - **부분 변수** 채움

* `patial`
    * 함수를 **부분적으로 사용**하는 것
    * 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우
        * 에시: **날짜**, **시간**

In [ ]:
# 오늘의 날짜를 구하는 파이썬 코드

from datetime import datetime

datetime.now().strftime("%B %d")                                 # 'August 01'

In [22]:
# 날짜 반환 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [ ]:
prompt_day = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today                                  # dictionary 형태로 partial_variables를 전달
    },
)

In [ ]:
# prompt 생성
prompt_day.format(n=3)

# '오늘의 날짜는 August 01 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [29]:
# chain 생성
chain_day = prompt_day | model2

In [ ]:
# chain 실행 후 결과 확인
print(chain_day.invoke(3).content)

<small>

*   
    * 셀 출력
        * 8월 1일 생일인 유명인 3명은 다음과 같습니다.  생년월일은 정확한 정보를 찾기 어려운 경우가 있어, 대략적인 정보를 제공합니다.

        * 1. **피터 오툴 (Peter O'Toole):**  (1932년 8월 2일 출생으로 알려져 있으나, 일부 자료에서는 8월 1일로 기재되어 있습니다.)  아일랜드 출신의 배우.  *로렌스 아라비아* 등의 영화로 유명합니다.

        * 2. **(추가 정보 필요):**  8월 1일 생일인 유명인을 두 명 더 찾기 위해서는 추가적인 정보가 필요합니다.  특정 분야 (예: 배우, 가수, 운동선수 등) 또는 국적을 지정해 주시면 더 정확한 정보를 제공할 수 있습니다.

        * 3. **(추가 정보 필요):**  위와 같이 추가 정보가 필요합니다.

        * 8월 1일은 많은 유명인의 생일이 아닌 날짜입니다.  더 구체적인 정보를 제공해주시면 더 도움을 드릴 수 있습니다.

In [ ]:
# chain 을 실행 후 결과를 확인합니다.
print(chain_day.invoke({"today": "Jan 02", "n": 3}).content)

<small>

* 
  * 셀 출력 
      * 오늘, 1월 2일이 생일인 유명인 3명과 그들의 생년월일은 다음과 같습니다.

      * 1.  **아이작 아시모프 (Isaac Asimov)**
          *   생년월일: 1920년 1월 2일

      * 2.  **폴 스미스 (Paul Smith)**
          *   생년월일: 1946년 1월 2일

      * 3.  **타이거 우즈 (Tiger Woods)**
          *   생년월일: 1975년 12월 30일 (참고: 타이거 우즈는 12월 30일생이지만, 1월 2일은 그의 생일과 가까운 날짜이므로 함께 언급될 수 있습니다. 만약 정확히 1월 2일생 유명인을 더 원하시면 다른 인물을 찾아드릴 수 있습니다.)

      * **참고:** 1월 2일생 유명인을 더 찾으신다면 알려주세요.

----

#### (4) **파일로부터 `template` 읽어오기**

In [ ]:
from langchain_core.prompts import load_prompt

prompt_yaml = load_prompt("prompts/fruit_color.yaml")
prompt_yaml

# PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [ ]:
prompt_yaml.format(fruit="사과")                            # '사과의 색깔이 뭐야?'

In [ ]:
prompt_yaml2 = load_prompt("prompts/capital.yaml")
print(prompt_yaml2.format(country="대한민국"))               

<small>

* 
  * 셀 출력 
      * 대한민국의 수도에 대해서 알려주세요.
      * 수도의 특징을 다음의 양식에 맞게 정리해 주세요.
      * 300자 내외로 작성해 주세요.
      * 한글로 작성해 주세요.
      * ----
      * [양식]
      * 1. 면적
      * 2. 인구
      * 3. 역사적 장소
      * 4. 특산품

      * #Answer:

---

#### **`ChatPromptTemplate`**

* 대화목록을 프롬프트로 주입하고자 할 때 활용
* 메시지는 **튜플** 형식으로 구성 
    * `role`, `message`
    * 리스트로 생성할 수 있음
* `role`
    * `system`: 시스템 설정 메시지 = 주로 **전역 설정과 관련된 메시지**
    * `human`: 사용자 입력 메시지
    * `ai`: AI 답변 메시지

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

<small>

* 
  * 셀 출력
      * ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [ ]:
chat_prompt.format(country="대한민국")                  # 'Human: 대한민국의 수도는 어디인가요?'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# chat message 생성
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

<small>

* 
  * 셀 출력
    * [SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

* 생성한 메시지를 바로 주입해 결과를 받을 수 있음

In [ ]:
model2.invoke(messages).content                     # '제 이름은 테디입니다. 만나서 반가워요!'

* chain 생성

In [ ]:
chain_chat = chat_template | model2

chain_chat.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

# '제 이름은 Teddy입니다. 만나서 반가워요!'

---

#### (6) **`MessagePlaceholder`**

* 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 공간
    * 메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않은 경우
    * 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

<small>

* 셀 출력
    * ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.FunctionMessageChunk, Tag(tag='FunctionMessageChunk')], typing.Annotated[langchain_core.messages.tool.ToolMessageChunk, Tag(tag='ToolMessageChunk')]], FieldInfo(annotation=NoneType, required=True, discriminator=Discriminator(discriminator=<function _get_type at 0x107e068e0>, custom_error_type=None, custom_error_message=None, custom_error_context=None))]]}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.'), additional_kwargs={}), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], input_types={}, partial_variables={}, template='지금까지의 대화를 {word_count} 단어로 요약합니다.'), additional_kwargs={})])

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

<small>

* 셀 출력
    * System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
    * Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
    * AI: 반가워요! 앞으로 잘 부탁 드립니다.
    * Human: 지금까지의 대화를 5 단어로 요약합니다.

In [ ]:
# chain 생성
chain_mph = chat_prompt | model2 | StrOutputParser()

# chain 실행 및 결과확인
chain_mph.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

# '새로운 동료, 환영, 인사'